In [28]:
library(tidyverse)
library(ggplot2)
library(patchwork)
source("helper.R")

In [29]:
sinfo_path = "../../output/sims_info.tsv"
sinfoh_path = "../../output/header_sims_info.tsv"

In [30]:
win_size = 1000000
win_len = -1
padding = 0
rescf = 1
prop_acc=0.4

In [31]:
fig_prefix = "cor-pidxy-dT_"

In [32]:
sims_path = "../../output/sims_results/"
varmut_sims_path = "../../output/varmut/sims_results"
data_path = paste0("../../output/greatapes-diversity-data/",fig_prefix,"win-size_",as.integer(win_size),"_merged-mask_True_prop-acc_",prop_acc, ".rds")

In [33]:
# loading sims info table
sinfo = read.table(sinfo_path, sep="\t")
h = read.table(sinfoh_path, sep="\t", header=TRUE)
cnames = colnames(h)
if (length(sinfo) == length(h)+1) {
    cnames = c(cnames, "poscoefdecayeps")
}
colnames(sinfo) = cnames

In [34]:
#diff_cols = c("delrate","delcoef","posrate","poscoef","mu", "rescf", "win_len", "padding", "chr")
diff_cols = c("delrate","delcoef","posrate","poscoef","mu", "poscoefdecayeps", "rescf", "win_len", "padding", "chr")

In [35]:
wsize_str = paste0("win-size_",as.integer(win_size))

In [51]:
sims_folders = paste0(sims_path,list.files(sims_path)[grep(wsize_str, list.files(sims_path))])

In [52]:
varmut_sims_folders = paste0(varmut_sims_path,list.files(varmut_sims_path)[grep(wsize_str, list.files(varmut_sims_path))])

In [53]:
sims_folders = c(sims_folders, varmut_sims_folders)

In [54]:
simresults = as.data.frame(do.call(rbind, lapply(X = sims_folders, function(x) unlist(meta_from_fname(x,prop=prop_acc,suffix="$")))), stringsAsFactors =FALSE)

[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"
[1] "entrei"


In [55]:
sims_folders

[1] "../../output/sims_results/sup-rand-id_0ASPD1FD6L57DMLSWJ_rep_0_win-size_1000000_sample-size_10"                                                            
  [2] "../../output/sims_results/sup-rand-id_1TRVVV3G18G1VCU5Y2_rep_0_win-size_1000000_sample-size_10"                                                            
  [3] "../../output/sims_results/sup-rand-id_1ZTIP09L94JNZTP6P1_rep_0_win-size_1000000_sample-size_10"                                                            
  [4] "../../output/sims_results/sup-rand-id_3CE1SNB9BZTMK1K8IE_rep_0_win-size_1000000_sample-size_10"                                                            
  [5] "../../output/sims_results/sup-rand-id_3GIHR79KVHKB6B02WF_rep_0_win-size_1000000_sample-size_10"                                                            
  [6] "../../output/sims_results/sup-rand-id_42LT1TYU611QINUWDG_rep_0_win-size_1000000_sample-size_10"                                                            
  [7] "../../output/sims_results/sup-rand-id_4FL6KFJXU6APHC2GA6_rep_0_win-size_1000000_sample-size_10"                                                            
  [8] "../../output/sims_results/sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10"                                                            
  [9] "../../output/sims_results/sup-rand-id_4NADO6705U5SN8I1V2_rep_0_win-size_1000000_sample-size_10"                                                            
 [10] "../../output/sims_results/sup-rand-id_520Y4OL8HJQN1FDSN9_rep_0_win-size_1000000_sample-size_10"                                                            
 [11] "../../output/sims_results/sup-rand-id_55RSB70EO1HGU1ZYFX_rep_0_win-size_1000000_sample-size_10"                                                            
 [12] "../../output/sims_results/sup-rand-id_562V5VDT6Y66UT3SKV_rep_0_win-size_1000000_sample-size_10"                                                            
 [13] "../../output/sims_results/sup-rand-id_57Y8L1U6MJBD98COMV_rep_0_win-size_1000000_sample-size_10"                                                            
 [14] "../../output/sims_results/sup-rand-id_5GTZF5LD803SRD6EQ9_rep_0_win-size_1000000_sample-size_10"                                                            
 [15] "../../output/sims_results/sup-rand-id_5IYZ1NWXLW8R8TR43Z_rep_0_win-size_1000000_sample-size_10"                                                            
 [16] "../../output/sims_results/sup-rand-id_5QXE1D7WE2XZIALIV8_rep_0_win-size_1000000_sample-size_10"                                                            
 [17] "../../output/sims_results/sup-rand-id_60X8ZD9TMU5E62ZOZA_rep_0_win-size_1000000_sample-size_10"                                                            
 [18] "../../output/sims_results/sup-rand-id_6HXVYZIAGCUED38QIL_rep_0_win-size_1000000_sample-size_10"                                                            
 [19] "../../output/sims_results/sup-rand-id_6MB8N72SPZ3RD6A5NC_rep_0_win-size_1000000_sample-size_10"                                                            
 [20] "../../output/sims_results/sup-rand-id_7RAE4BNCELKNBN2LQ8_rep_0_win-size_1000000_sample-size_10"                                                            
 [21] "../../output/sims_results/sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10"                                                            
 [22] "../../output/sims_results/sup-rand-id_8NYARZTMLXU8JGXY6O_rep_0_win-size_1000000_sample-size_10"                                                            
 [23] "../../output/sims_results/sup-rand-id_8VJOM0D3I01B2NRYJT_rep_0_win-size_1000000_sample-size_10"                                                            
 [24] "../../output/sims_results/sup-rand-id_93GIWTX2GJNEC8V18B_rep_0_win-size_1000000_sample-size_10"                                                            
 [25] "../../output/sims_results/sup-rand-id_9H89YU146KRYEE6CEL_rep_0_win-size_1000000_sam

In [56]:
simresults$folderpath = sims_folders

In [57]:
tmp = unique(sinfo[sinfo$sup_rand_id %in% simresults$sup_rand_id,c(diff_cols,"sup_rand_id")])

In [69]:
sinfo[sinfo$sup_rand_id=="O814WK8MN3UOF27JIR",]

,edge,parent,N,gens,rescf,padding,delrate,delcoef,posrate,poscoef,⋯,supnumid,sup_rand_id,outfile_pre,exonfile,recfile,recfilehap,par_string,date,log,poscoefdecayeps
,<fct>,<fct>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
131,western-gorila,gorilla,20132,7895,1,0,6e-09,-0.015,0,0,⋯,24,O814WK8MN3UOF27JIR,../../output/6DRX6RL6BNZCHP8/western-gorila_6DRX6RL6BNZCHP8,../../output/maps/6DRX6RL6BNZCHP8_exons.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.hapmap,"-d recfile=\""../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv\"" -d exonfile=\""../../output/maps/6DRX6RL6BNZCHP8_exons.tsv\"" -d siminterval=\""\"" -d L=\""132000000\"" -d mu=\""6e-09\"" -d delprop=\""100.0\"" -d delcoef=\""-0.015\"" -d posprop=\""0.0\"" -d poscoef=\""0.0\"" -d N=\""20132\"" -d gens=\""7895\"" -d rescf=\""1\""",1046_30112021,../../output/tmp/western-gorila_6DRX6RL6BNZCHP8.log,NA
2558,central-chimp,eastern-central,30000,7000,1,0,6e-09,-0.015,0,0,⋯,24,O814WK8MN3UOF27JIR,../../output/6DRX6RL6BNZCHP8/central-chimp_6DRX6RL6BNZCHP8,../../output/maps/6DRX6RL6BNZCHP8_exons.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.hapmap,"-d recfile=\""../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv\"" -d exonfile=\""../../output/maps/6DRX6RL6BNZCHP8_exons.tsv\"" -d siminterval=\""\"" -d L=\""132000000\"" -d mu=\""6e-09\"" -d delprop=\""100.0\"" -d delcoef=\""-0.015\"" -d posprop=\""0.0\"" -d poscoef=\""0.0\"" -d N=\""30000\"" -d gens=\""7000\"" -d rescf=\""1\""",1054_30112021,../../output/tmp/central-chimp_6DRX6RL6BNZCHP8.log,NA
3323,western-chimp,nigerian-western,5100,9400,1,0,6e-09,-0.015,0,0,⋯,24,O814WK8MN3UOF27JIR,../../output/6DRX6RL6BNZCHP8/western-chimp_6DRX6RL6BNZCHP8,../../output/maps/6DRX6RL6BNZCHP8_exons.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.hapmap,"-d recfile=\""../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv\"" -d exonfile=\""../../output/maps/6DRX6RL6BNZCHP8_exons.tsv\"" -d siminterval=\""\"" -d L=\""132000000\"" -d mu=\""6e-09\"" -d delprop=\""100.0\"" -d delcoef=\""-0.015\"" -d posprop=\""0.0\"" -d poscoef=\""0.0\"" -d N=\""5100\"" -d gens=\""9400\"" -d rescf=\""1\""",1057_30112021,../../output/tmp/western-chimp_6DRX6RL6BNZCHP8.log,NA
5056,great-apes,,125089,250178,1,0,6e-09,-0.015,0,0,⋯,24,O814WK8MN3UOF27JIR,../../output/6DRX6RL6BNZCHP8/great-apes_6DRX6RL6BNZCHP8,../../output/maps/6DRX6RL6BNZCHP8_exons.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.hapmap,"-d recfile=\""../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv\"" -d exonfile=\""../../output/maps/6DRX6RL6BNZCHP8_exons.tsv\"" -d siminterval=\""500\"" -d L=\""132000000\"" -d mu=\""6e-09\"" -d delprop=\""100.0\"" -d delcoef=\""-0.015\"" -d posprop=\""0.0\"" -d poscoef=\""0.0\"" -d N=\""125089\"" -d gens=\""250178\"" -d rescf=\""1\""",1104_30112021,../../output/tmp/great-apes_6DRX6RL6BNZCHP8.log,NA
5117,orangutans,great-apes,19231,409731,1,0,6e-09,-0.015,0,0,⋯,24,O814WK8MN3UOF27JIR,../../output/6DRX6RL6BNZCHP8/orangutans_6DRX6RL6BNZCHP8,../../output/maps/6DRX6RL6BNZCHP8_exons.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.hapmap,"-d recfile=\""../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv\"" -d exonfile=\""../../output/maps/6DRX6RL6BNZCHP8_exons.tsv\"" -d siminterval=\""\"" -d L=\""132000000\"" -d mu=\""6e-09\"" -d delprop=\""100.0\"" -d delcoef=\""-0.015\"" -d posprop=\""0.0\"" -d poscoef=\""0.0\"" -d N=\""19231\"" -d gens=\""409731\"" -d rescf=\""1\""",1104_30112021,../../output/tmp/orangutans_6DRX6RL6BNZCHP8.log,NA
5649,nigerian-western,chimps,10100,7720,1,0,6e-09,-0.015,0,0,⋯,24,O814WK8MN3UOF27JIR,../../output/6DRX6RL6BNZCHP8/nigerian-western_6DRX6RL6BNZCHP8,../../output/maps/6DRX6RL6BNZCHP8_exons.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.tsv,../../output/maps/6DRX6RL6BNZCHP8_recrate.hapmap,"-d recfile=\""../../ou

In [60]:
simresults = left_join(simresults, tmp)

Joining, by = "sup_rand_id"



In [62]:
simresults = simresults[simresults$win_len==win_len & simresults$padding == padding & simresults$rescf==1 & as.numeric(simresults$win_size) == win_size,]

In [63]:
win_size

[1] 1e+06

In [64]:
simresults

,win_size,sup_rand_id,rep,sample_size,spaced_desc,desc,prop,is_ga_data,sigma,folderpath,delrate,delcoef,posrate,poscoef,mu,poscoefdecayeps,rescf,win_len,padding,chr
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<fct>
5,1000000,3GIHR79KVHKB6B02WF,0,10,sup-rand-id=3GIHR79KVHKB6B02WF rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_3GIHR79KVHKB6B02WF_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_3GIHR79KVHKB6B02WF_rep_0_win-size_1000000_sample-size_10,6.0e-09,-0.030,1e-10,0.01,6.1000e-09,NA,1,-1,0,chr12
7,1000000,4FL6KFJXU6APHC2GA6,0,10,sup-rand-id=4FL6KFJXU6APHC2GA6 rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_4FL6KFJXU6APHC2GA6_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_4FL6KFJXU6APHC2GA6_rep_0_win-size_1000000_sample-size_10,1.2e-08,-0.030,1e-11,0.01,1.2010e-08,0.001,1,-1,0,chr12
8,1000000,4GROMS6FHMNW5X9L72,0,10,sup-rand-id=4GROMS6FHMNW5X9L72 rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10,0.0e+00,0.000,1e-11,0.01,1.0000e-11,NA,1,-1,0,chr12
11,1000000,55RSB70EO1HGU1ZYFX,0,10,sup-rand-id=55RSB70EO1HGU1ZYFX rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_55RSB70EO1HGU1ZYFX_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_55RSB70EO1HGU1ZYFX_rep_0_win-size_1000000_sample-size_10,2.0e-09,-0.015,1e-10,0.01,2.1000e-09,NA,1,-1,0,chr12
21,1000000,8MF0I020MWGI6N41PH,0,10,sup-rand-id=8MF0I020MWGI6N41PH rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10,0.0e+00,0.000,0e+00,0.00,0.0000e+00,NA,1,-1,0,chr12
26,1000000,9WZ486SOY2Q604EOMM,0,10,sup-rand-id=9WZ486SOY2Q604EOMM rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_9WZ486SOY2Q604EOMM_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_9WZ486SOY2Q604EOMM_rep_0_win-size_1000000_sample-size_10,1.2e-08,-0.030,1e-12,0.01,1.2001e-08,0.001,1,-1,0,chr12
27,1000000,AA9GUSRHWOA341WGC4,0,10,sup-rand-id=AA9GUSRHWOA341WGC4 rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_AA9GUSRHWOA341WGC4_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_AA9GUSRHWOA341WGC4_rep_0_win-size_1000000_sample-size_10,1.2e-08,-0.030,0e+00,0.00,1.2000e-08,NA,1,-1,0,chr12
34,1000000,CR1UIIF8NRSPZ94LGF,0,10,sup-rand-id=CR1UIIF8NRSPZ94LGF rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_CR1UIIF8NRSPZ94LGF_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_CR1UIIF8NRSPZ94LGF_rep_0_win-size_1000000_sample-size_10,1.4e-08,-0.030,0e+00,0.00,1.4000e-08,NA,1,-1,0,chr12
38,1000000,DFFB9J9GYC00WIFTAQ,0,10,sup-rand-id=DFFB9J9GYC00WIFTAQ rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_DFFB9J9GYC00WIFTAQ_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_DFFB9J9GYC00WIFTAQ_rep_0_win-size_1000000_sample-size_10,1.4e-08,-0.030,1e-11,0.01,1.4010e-08,NA,1,-1,0,chr12


In [65]:
simresults[is.na(simresults$poscoefdecayeps),]$poscoefdecayeps = 0

In [66]:
simresults$selregime = ifelse(simresults$posrate<1e-16,ifelse(simresults$delrate<1e-16, ifelse(simresults$sigma==0, "neutral", "varmut"), "del_only"), ifelse(simresults$delrate<1e-16, "pos_only", "both"))

In [67]:
simresults$seldesc = apply(simresults,1, function(x) paste("mu-D=", x["delrate"], " mu-B=", x["posrate"], "\ncoef-D=", x["delcoef"], " coef-B=", x["poscoef"], " decay-B=", x["poscoefdecayeps"], sep=""))

In [68]:
simresults

,win_size,sup_rand_id,rep,sample_size,spaced_desc,desc,prop,is_ga_data,sigma,folderpath,⋯,posrate,poscoef,mu,poscoefdecayeps,rescf,win_len,padding,chr,selregime,seldesc
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<fct>,<chr>,<chr>
5,1000000,3GIHR79KVHKB6B02WF,0,10,sup-rand-id=3GIHR79KVHKB6B02WF rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_3GIHR79KVHKB6B02WF_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_3GIHR79KVHKB6B02WF_rep_0_win-size_1000000_sample-size_10,⋯,1e-10,0.01,6.1000e-09,0.000,1,-1,0,chr12,both,mu-D=6.0e-09 mu-B=1e-10 coef-D=-0.030 coef-B=0.01 decay-B=0.000
7,1000000,4FL6KFJXU6APHC2GA6,0,10,sup-rand-id=4FL6KFJXU6APHC2GA6 rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_4FL6KFJXU6APHC2GA6_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_4FL6KFJXU6APHC2GA6_rep_0_win-size_1000000_sample-size_10,⋯,1e-11,0.01,1.2010e-08,0.001,1,-1,0,chr12,both,mu-D=1.2e-08 mu-B=1e-11 coef-D=-0.030 coef-B=0.01 decay-B=0.001
8,1000000,4GROMS6FHMNW5X9L72,0,10,sup-rand-id=4GROMS6FHMNW5X9L72 rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_4GROMS6FHMNW5X9L72_rep_0_win-size_1000000_sample-size_10,⋯,1e-11,0.01,1.0000e-11,0.000,1,-1,0,chr12,pos_only,mu-D=0.0e+00 mu-B=1e-11 coef-D= 0.000 coef-B=0.01 decay-B=0.000
11,1000000,55RSB70EO1HGU1ZYFX,0,10,sup-rand-id=55RSB70EO1HGU1ZYFX rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_55RSB70EO1HGU1ZYFX_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_55RSB70EO1HGU1ZYFX_rep_0_win-size_1000000_sample-size_10,⋯,1e-10,0.01,2.1000e-09,0.000,1,-1,0,chr12,both,mu-D=2.0e-09 mu-B=1e-10 coef-D=-0.015 coef-B=0.01 decay-B=0.000
21,1000000,8MF0I020MWGI6N41PH,0,10,sup-rand-id=8MF0I020MWGI6N41PH rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_8MF0I020MWGI6N41PH_rep_0_win-size_1000000_sample-size_10,⋯,0e+00,0.00,0.0000e+00,0.000,1,-1,0,chr12,neutral,mu-D=0.0e+00 mu-B=0e+00 coef-D= 0.000 coef-B=0.00 decay-B=0.000
26,1000000,9WZ486SOY2Q604EOMM,0,10,sup-rand-id=9WZ486SOY2Q604EOMM rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_9WZ486SOY2Q604EOMM_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_9WZ486SOY2Q604EOMM_rep_0_win-size_1000000_sample-size_10,⋯,1e-12,0.01,1.2001e-08,0.001,1,-1,0,chr12,both,mu-D=1.2e-08 mu-B=1e-12 coef-D=-0.030 coef-B=0.01 decay-B=0.001
27,1000000,AA9GUSRHWOA341WGC4,0,10,sup-rand-id=AA9GUSRHWOA341WGC4 rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_AA9GUSRHWOA341WGC4_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_AA9GUSRHWOA341WGC4_rep_0_win-size_1000000_sample-size_10,⋯,0e+00,0.00,1.2000e-08,0.000,1,-1,0,chr12,del_only,mu-D=1.2e-08 mu-B=0e+00 coef-D=-0.030 coef-B=0.00 decay-B=0.000
34,1000000,CR1UIIF8NRSPZ94LGF,0,10,sup-rand-id=CR1UIIF8NRSPZ94LGF rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_CR1UIIF8NRSPZ94LGF_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_CR1UIIF8NRSPZ94LGF_rep_0_win-size_1000000_sample-size_10,⋯,0e+00,0.00,1.4000e-08,0.000,1,-1,0,chr12,del_only,mu-D=1.4e-08 mu-B=0e+00 coef-D=-0.030 coef-B=0.00 decay-B=0.000
38,1000000,DFFB9J9GYC00WIFTAQ,0,10,sup-rand-id=DFFB9J9GYC00WIFTAQ rep=0 win-size=1000000 sample-size=10 prop-acc=0.4,sup-rand-id_DFFB9J9GYC00WIFTAQ_rep_0_win-size_1000000_sample-size_10_prop-acc_0.4,0.4,FALSE,0,../../output/sims_results/sup-rand-id_DFFB9J9GYC00WIFTAQ_rep_0_win-size_1000000_sample-size_10,⋯,1e-11,0.01,1.4010e-08,0.000,1,-1,0,chr12,bot

In [ ]:
tmp = simresults
tmp = tmp[
  with(tmp, order(posrate,-poscoefdecayeps,delrate,poscoef,delcoef)),
]
rownames(tmp) <- NULL
plots = apply(tmp,1,
              function(x) readRDS(paste0(sims_path,x["folderpath"],
                                         "/",fig_prefix,x["desc"],".rds"))+ggtitle(x["seldesc"]))
#plots = c(readRDS(data_path)+ggtitle("data"), plots)

In [ ]:
tmp

In [ ]:
plots=c(list(readRDS(data_path)+ggtitle("data")), plots)

In [ ]:
ordered = c("neutral", "del_only", "pos_only", "both")

In [ ]:
classes = unique(tmp$selregime)
tmp$selregime = factor(tmp$selregime, levels=classes[order(match(classes,ordered))])

In [ ]:
tab=table(tmp$selregime)

In [ ]:
tab = c("data"=1, tab)

In [ ]:
tab

In [ ]:
areas = list()
row = 1
for (regime in names(tab)) {
    col = 1
    for(i in 1:tab[regime]) {
        areas = c(areas, list(area(row,col)))
        col = col + 1
    }
    row = row + 1
}

In [ ]:
areas = (do.call(c, areas))

In [ ]:
#design=""
#for (regime in names(tab)) {
#    if (design != "") design = paste0(design,"\n")
#    num_cols = 1:tab[regime]
#    num_blanks = max(tab)-tab[regime]
#    if (regime == "data") {
#        indexes = c(1)
#    } else {
#        indexes = 1+as.numeric(rownames(tmp[tmp$selregime==regime,]))        
#    }
#    print(LETTERS[indexes])
#    print(rep("#", num_blanks))
#    add = paste0(paste0(LETTERS[indexes],collapse=""), paste0(rep("#", num_blanks),collapse=""), collapse="")
#    print(design)
#    print(add)
#    design = paste0(design,add)
#    print(design)
#}

In [ ]:
panel_plot = wrap_plots(plots, nrow=length(classes), design=areas, guides="collect")

In [ ]:
ggsave(filename=paste0("../../output/",fig_prefix,"panel_sims_and_data_win-size_",win_size,".pdf"), plot=panel_plot, width=100, height=25, scale=1.75, units="cm", limitsize=FALSE)

In [ ]:
plots[[1]]+ggtitle("lala")

In [ ]:
tmp